# Generate Infinite Tiktoks

## Generate video text
The first step to generate a tiktok, is obviously to know what you're going to stay.

To do so, my idea was to find something in genius.com, the famous website to have extra information on rap/pop songs. You can have informations about the context of the song when it was released.


In [1]:
import os
import requests


In [2]:
def get_annotation(annotation_id):
    url = f"https://api.genius.com/annotations/{annotation_id}"
    payload = {}
    headers = {'Authorization': 'Bearer ' + os.environ['RAPGENIUS_API_KEY']}
    r = requests.request("GET", url, headers=headers, data=payload)
    data = r.json()
    
    full_annotation = ""
    parts = data['response']['annotation']['body']['dom']['children']

    full_annotation = find_annotation(parts)
    
    return full_annotation
def find_annotation(root):
    # print(type(root))
    full_annotation = ""
    if(type(root) == str):
        if(root == ""):
            root = '\n'
        full_annotation += root
    elif(type(root) == dict):
        if('children' in root):
            full_annotation += find_annotation(root['children'])
    elif(type(root) == list):
        for part in root:
            full_annotation += find_annotation(part)
    
    return full_annotation
def get_id_song(url):
    response = requests.get(url).content
    id = int(response.split(b'genius://songs/')[1].split(b'"')[0])
    return id
def get_song_description(song_id):
    url = f"https://api.genius.com/songs/{song_id}"

    payload = {}
    headers = {
        'Authorization': 'Bearer ' + os.environ['RAPGENIUS_API_KEY']
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    data = response.json()
    return get_annotation(data['response']['song']['description_annotation']['id'])


In [3]:
url = "https://genius.com/Eminem-houdini-lyrics"

song_description = get_song_description(get_id_song(url))
print(song_description)

“Houdini” is the lead single of Eminem’s twelfth studio album The Death of Slim Shady (Coup de Grâce). The track is named after Harry Houdini, a popular magician known for his death-defying stunts like the Chinese Water Torture Cell, which Eminem replicates in the intro to his 1999 track, “Role Model.”
With the return of long-time collaborator Jeff Bass of the Bass Brothers, Eminem revisits his catchy albeit morbid sound of the early 2000s. This fanfare track sees him callback to his old catchphrases such as Shady’s iconic “Guess who’s back?” line from the 2002 track “Without Me.” The song heavily samples and interpolates the Steve Miller Band’s 1982 track “Abracadabra.”
On October 24, 1926, the legendary Houdini tragically passed away in Detroit, Michigan at the age of 52. Interestingly, Detroit is also the hometown of Eminem, who turns 52 in October 2024.
The single was first teased by Eminem on May 20, 2024, when he posted a teaser to his social media accounts with a text message fr

## Find pictures

### Find picture descrition
The idea I came up with is to ask a LLM (mistral in my case) to find some nice moments in the description of the song

In [24]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

def get_description_pictures(story):
    api_key = os.environ["MISTRAL_API_KEY"]
    model = "open-mistral-7b"

    client = MistralClient(api_key=api_key)

    preprompt = """Your job will be to write the visual of a tiktok video: the images that show on screen. You have to find some nice moments in the story to place pictures that will show on screen. Tell me when you choose an image of what. In this format:
    DURING THE SENTENCE "[place the full sentence here]", SHOW AN IMAGE OF "[place the description of the image]".
    Answer only with the descriptions of all the images. 
    You can set as many images as you want, and if a sentence needs more than one image, you can split the sentence in two or more parts.
    Try not to give a description, but key words because i will look online for the exact same words you describe. That means if you are too complex, i will not be able to find the image.
    
    For example:
    DURING THE SENTENCE "he ate a sandwich", SHOW AN IMAGE OF "a sandwich".

    Now it is your turn:

    STORY : """

    messages = [
        ChatMessage(role="user", content=preprompt + f'"{story}"'),
    ]

    chat_response = client.chat(
        model=model,
        messages=messages,
    )

    results = chat_response.choices[0].message.content

    return results.split("\n\n")
def convert_to_dict(lines):
    visual_dict = {}
    for line in lines:
        print(line)
        if line:
            sentence, image_desc = line.split(', SHOW AN IMAGE OF ', 1)
            sentence = sentence.replace('DURING THE SENTENCE ', '', 1)
            visual_dict[sentence] = image_desc
    return visual_dict

def get_pictures(story):
    return convert_to_dict(get_description_pictures(story))


In [25]:
descriptions = get_pictures(song_description)
print(descriptions)

DURING THE SENTENCE "Houdini, a popular magician known for his death-defying stunts like the Chinese Water Torture Cell", SHOW AN IMAGE OF "Chinese Water Torture Cell".
DURING THE SENTENCE "Eminem replicates in the intro to his 1999 track, “Role Model,”", SHOW AN IMAGE OF "Eminem in the intro of Role Model (1999)".
DURING THE SENTENCE "With the return of long-time collaborator Jeff Bass of the Bass Brothers", SHOW AN IMAGE OF "Jeff Bass and Eminem".
DURING THE SENTENCE "The song heavily samples and interpolates the Steve Miller Band’s 1982 track “Abracadabra”", SHOW AN IMAGE OF "Steve Miller Band's Abracadabra album cover".
DURING THE SENTENCE "On October 24, 1926, the legendary Houdini tragically passed away in Detroit, Michigan at the age of 52", SHOW AN IMAGE OF "Houdini's death scene or his grave".
DURING THE SENTENCE "Interestingly, Detroit is also the hometown of Eminem, who turns 52 in October 2024", SHOW AN IMAGE OF "Detroit cityscape or Eminem in Detroit".
DURING THE SENTENCE 

### Generate or Find picture

In [26]:
from google_images_search import GoogleImagesSearch

def get_first_image_url(search_term, path_to_dir='images'):
    search_term = search_term.replace('"', '')
    # print(search_term)
    gis = GoogleImagesSearch(os.environ['GOOGLE_API_KEY'] , os.environ['GOOGLE_CX_KEY'])
    _search_params = {
        'q': search_term
    }
    gis.search(search_params=_search_params, path_to_dir=path_to_dir)

    for result in gis.results():
        return result.url

    return None

In [28]:
imgs_url = {}
for key in descriptions:
    # print(key)
    # print(descriptions[key])
    imgs_url[key] = get_first_image_url(descriptions[key], None)
    # break
print(imgs_url)

"Chinese Water Torture Cell".
Chinese Water Torture Cell.
"Eminem in the intro of Role Model (1999)".
Eminem in the intro of Role Model (1999).
"Jeff Bass and Eminem".
Jeff Bass and Eminem.
"Steve Miller Band's Abracadabra album cover".
Steve Miller Band's Abracadabra album cover.
"Houdini's death scene or his grave".
Houdini's death scene or his grave.
"Detroit cityscape or Eminem in Detroit".
Detroit cityscape or Eminem in Detroit.
"Eminem's social media post teasing the single".
Eminem's social media post teasing the single.
"Eminem on FaceTime with David Blaine".
Eminem on FaceTime with David Blaine.
"Eminem announcing the release date".
Eminem announcing the release date.
"Eminem's The Death of Slim Shady (Coup de Grâce) album cover".
Eminem's The Death of Slim Shady (Coup de Grâce) album cover.
{'"Houdini, a popular magician known for his death-defying stunts like the Chinese Water Torture Cell"': 'https://upload.wikimedia.org/wikipedia/commons/e/e8/Houdini_performing_Water_Tortu

## Create video from the pictures